<a href="https://colab.research.google.com/github/ratan-asthana/ratan-repositiory/blob/master/Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
#shop_data = pd.read_csv("C:/Users/Kaggle_Data/Extracted_Data/shops.csv")

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()

In [0]:
pd.read_csv('test.csv')

In [0]:
pd.read_csv('items.csv')

In [0]:
shop_data = pd.read_csv('shops.csv')

In [0]:
item_data = pd.read_csv('items.csv')

In [0]:
item_categories = pd.read_csv('item_categories.csv')

In [0]:
train_data = pd.read_csv('sales_train_v2.csv' , parse_dates=['date'])

In [0]:
train_data.head()

In [0]:
test_data = pd.read_csv('test.csv')

In [0]:
test_data.head()

In [0]:
item_categories.head()

In [0]:
# joining the tables 

'''
here we are joining the data from different sources , the rsuffix is being used in case overlapping of similar columns happen then 
 there should not be any value error so it will append the '_' towards the common value
 later on in the drop command we are dropping the similar values
'''

training_set = train_data.join(item_data , on='item_id' , rsuffix='_').join(shop_data , on='shop_id' , rsuffix='_').join(item_categories , on='item_category_id' , rsuffix='_').drop(['item_id_' , 'shop_id_','item_category_id_'] , axis=1)

In [0]:
training_set.head()

In [0]:
# total rows and columns calculation

print(" Rows : " ,training_set.shape[0])
print(" Columns : " ,training_set.shape[1])

In [0]:
# checking the dates - max and min dates 

print("Min date " , training_set['date'].min().date())
print("Max date " , training_set['date'].max().date())

In [0]:
#print('Months Difference : ' , )

from datetime import datetime
from dateutil import relativedelta
date1 = datetime.strptime('2013-01-01 ', '%Y-%m-%d ')
date2 = datetime.strptime('2015-12-10', '%Y-%m-%d')
r = relativedelta.relativedelta(date2, date1)
r.months * (r.years+1)

In [0]:
# shop is in test data set 

test_shop_ids = test_data['shop_id'].unique()

# items in test data set 

test_item_ids = test_data['item_id'].unique()

In [0]:
#test_data.head()  checking the columns in test data

useful_training_set = train_data[train_data['shop_id'].isin(test_shop_ids)]

In [0]:
useful_training_set = useful_training_set[useful_training_set['item_id'].isin(test_item_ids)]

In [0]:
print('Initial Data ' , train_data.shape[0])
print('Useful Data ' , useful_training_set.shape[0])

In [0]:
train_data_0_price = train_data.query('item_price == 0')
print(train_data_0_price)

# so there is no item with 0 price

In [0]:
train_data_negative_price = train_data.query('item_price < 0')
print(train_data_negative_price)

# there is a single record with negative price

In [0]:
selected_data = train_data.query('item_price > 0 ')
print(selected_data.count())

# so this is the selected data on which work is to be done

In [0]:
training_set = useful_training_set[['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day']]

In [0]:
training_set = training_set.sort_values('date').groupby(['date_block_num', 'shop_id','item_id'] , as_index=False)

In [0]:
training_set = training_set.agg({'item_price':['sum' , 'mean'] , 'item_cnt_day':['sum' , 'mean' , 'count']})

In [0]:
training_set.columns = ['date_block_num', 'shop_id', 'item_id', 'item_price', 'mean_item_price', 'item_cnt', 'mean_item_cnt', 'transactions']

training_set.head()

In [0]:
type(training_set)

shop_ids = training_set['shop_id'].unique()
item_ids = training_set['item_id'].unique()
print(shop_ids)
print(item_ids)

In [0]:
empty_df = []
for i in range(33):
  for shop in shop_ids:
    for item in item_ids:
      empty_df.append([i,shop,item])


empty_df = pd.DataFrame(empty_df,columns=['date_block_num' , 'shop_id' , 'item_id'])
empty_df.head()

#training_set.plot(x='item_price' , y='item_cnt_day' , kind='scatter')

# it should be of same size

#training_set.columns = 

#training_set.head()


In [0]:
# merging the train set with complete set

# merge syntax : pandas.merge(left_dataframe , right_dataframe , left_on(column of left dataframe) , right_on(column of right dataframe) , how(join type like inner , left))


training_set = pd.merge(empty_df , training_set , on=['date_block_num' , 'shop_id' , 'item_id'] , how='left')
training_set.fillna(0 , inplace=True)

In [0]:
training_set.head().T

In [0]:
training_set['year'] = training_set['date_block_num'].apply(lambda x: ((x//12) + 2013 ))
training_set['monthly'] = training_set['date_block_num'].apply(lambda x: (x%12))

In [0]:
# grouping data for EDA - 

#grp_month_mean = training_set.groupby(['monthly'] , as_index=False)['item_price, mean'].mean()

grp_month_mean = training_set.groupby(['monthly'] , as_index=False)['item_cnt'].mean()
grp_month_sum = training_set.groupby(['monthly'] , as_index=False)['item_cnt'].sum()
grp_shop_mean = training_set.groupby(['shop_id'] , as_index=False)['item_cnt'].mean()
grp_shop_sum = training_set.groupby(['shop_id'] , as_index=False)['item_cnt'].sum()



In [0]:
import seaborn as sns
f , axes = plt.subplots(2 , 1 , figsize=(22,10) , sharex=True)
sns.lineplot(x="monthly" , y="item_cnt" , data = grp_month_mean , ax=axes[0]).set_title("Monthly Mean")
sns.lineplot(x="monthly" , y="item_cnt" , data = grp_month_sum , ax=axes[1]).set_title("Monthly Sum")
plt.show()

In [0]:
f , axes = plt.subplots(2,1, figsize=(22,10),sharex=True)
sns.barplot(x="shop_id" , y="item_cnt" , data=grp_shop_mean , ax=axes[0] , palette="rocket" ).set_title("Monthly Mean")
sns.barplot(x="shop_id" , y="item_cnt" , data=grp_shop_sum , ax=axes[1] , palette="rocket" ).set_title("Monthly Sum")
plt.show()

In [0]:
# checking outliers 

#sns.jointplot(x="item_cnt", y="item_price", data=train_monthly, height=8)
# boxplot
# seaborn.boxplot(x=None, y=None, hue=None, data=None, order=None, hue_order=None, orient=None, color=None, 
#palette=None, saturation=0.75, width=0.8, dodge=True, fliersize=5, linewidth=None, whis=1.5, notch=False, ax=None, **kwargs

sns.boxplot(x="item_cnt" , y="item_price" , data=training_set , linewidth=1.5 )
plt.show()

In [0]:
sns.jointplot(x="item_cnt", y="item_price", data=training_set, height=8)
plt.show()

In [0]:
sns.boxplot(training_set['item_cnt'])
plt.show()

In [0]:
sns.boxplot(training_set['item_price'])
plt.show()